In [ ]:
#als nächste: die jahre berücktrichten!!!!!

In [4]:
import os
import pandas as pd
import json
import requests
from meteostat import Daily, Stations
from datetime import datetime
from data_preparation import download_files, files, output_folder
from data_preparation import data_import, data_cleaning, weather_data, geo_data, convert_date


In [ ]:
import data_preparation
print(dir(data_preparation))

In [5]:
df=data_import()
df.head()

Spalte 'Specie' fehlt in airquality-covid19-cities.json
waqi-covid-2022Q1.csv enthält nach Duplikat-Entfernung keine Daten mehr.


,Date,Country,City,Specie,count,min,max,median,variance
0,2015-01-06,KR,Jeonju,co,124,0.1,12.3,4.5,55.74
1,2015-01-22,KR,Jeonju,co,116,4.5,10.0,6.7,16.09
2,2015-03-30,KR,Jeonju,co,118,1.2,11.2,5.6,35.98
3,2015-05-27,KR,Jeonju,co,93,2.3,5.6,3.4,6.54
4,2015-02-03,KR,Jeonju,co,133,4.5,13.4,7.8,39.24


In [6]:
import data_preparation
print(dir(data_preparation))
print(os.getcwd())

['Daily', 'Stations', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', 'convert_date', 'data_cleaning', 'data_import', 'datetime', 'download_files', 'files', 'geo_data', 'json', 'os', 'output_folder', 'pd', 'population_data', 'requests', 'weather_data']
c:\Users\whypk\01Projekte\air_quality


In [7]:
df.head()   

,Date,Country,City,Specie,count,min,max,median,variance
0,2015-01-06,KR,Jeonju,co,124,0.1,12.3,4.5,55.74
1,2015-01-22,KR,Jeonju,co,116,4.5,10.0,6.7,16.09
2,2015-03-30,KR,Jeonju,co,118,1.2,11.2,5.6,35.98
3,2015-05-27,KR,Jeonju,co,93,2.3,5.6,3.4,6.54
4,2015-02-03,KR,Jeonju,co,133,4.5,13.4,7.8,39.24


In [10]:
def data_cleaning2(df):
    """Bereinigung der Daten
    - nicht benötigte Spalten löschen
    - Zusammenfassung der Daten nach Datum, Land, Stadt und Spezies, so dass nur ein Messwert je Species (Median) pro Tag/ Stadt verbleibt
    - Spalte Species aufteilen
    - df als csv speichern im Datenverzeichnis
    """
    df = df.copy()

    df = df.drop(columns=['variance', 'min', 'max'], errors='ignore')

    df = df.groupby(["Date", "Country", "City", "Specie"], as_index=False).agg({"median": "mean"})  

    df = df.pivot(index=["Date", "Country", "City"], columns="Specie", values='median').reset_index()

    df["City"] = df["City"].str.lower().str.strip()
    df['Date'] = pd.to_datetime(df['Date']).dt.strftime('%Y-%m-%d')
    #df.columns = df.columns.str.strip().str.lower()

    #df = geo_data(df)

    #df = weather_data(df)

    
    df['Date'] = pd.to_datetime(df['Date']) # Convert 'Date' column to datetime
    # Split 'Date' column into 'year', 'month' and 'day'
    df['year'] = df['Date'].dt.year
    df['month'] = df['Date'].dt.month
    df['day'] = df['Date'].dt.day
    if 'Date' in df.columns:            # Remove 'Date' column
        df.drop(columns=['Date'], inplace=True)

    output_path = './data/cleaned_data.csv'
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    df.to_csv(output_path, index=False)
    print(f"✅ Datei wurde gespeichert: {output_path}")
    
    return(df)

In [11]:
df=data_cleaning2(df)
df.head()

AttributeError: 'NoneType' object has no attribute 'copy'

In [ ]:
def convert_date(df):
    """
    Teilt die Spalte Date in Year, Month, Day auf
    """
    # Convert 'Date' column to datetime
    df['Date'] = pd.to_datetime(df['Date'])

    # Split 'Date' column into 'year', 'month' and 'day'
    df['year'] = df['Date'].dt.year
    df['month'] = df['Date'].dt.month
    df['day'] = df['Date'].dt.day

    # Remove 'Date' column
    if 'Date' in df.columns:
        df.drop(columns=['Date'], inplace=True)

In [14]:
df=data_cleaning(df)
df.head()

KeyError: 'City'

In [ ]:
df_population = pd.read_csv(
    '/Users/whypk/01Projekte/air_quality/data/population.csv',
    sep=',', 
    header=0,
    engine='python',  # Hilft oft bei Parsing-Problemen
    on_bad_lines='skip',  # Methode zum Überspringen von fehlerhaften Zeilen
    encoding='utf-8'  # Falls Sonderzeichen vorhanden sind
)

In [ ]:
print(df_population.head())
print(df_population.columns)
print(df_population.dtypes)
print(df_population.shape)

In [ ]:
df_population= df_population[['City', 'Value', 'Year']]
df_population.head()


In [ ]:

#df_population['population'] = df_population['population'].astype(int)
#df_population["City"] = df_population["City"].str.lower().str.strip()
df_population.rename(columns={'Value': 'Population'}, inplace=True)
df_population.head()

In [ ]:
df_population.dropna(inplace=True)
df_population['Population'] = df_population['Population'].astype(int)
df_population.head()

In [ ]:
df_ = pd.merge(df, df_population, on=['City', 'Year'], how='left')

In [ ]:
def population_data(df):
    '''
    Fügt dem jeder Stadt Einwohner hinzu
    '''
    df = df.copy()
    
    df_population = pd.read_csv(
                    './data/population.csv',
                    sep=',', 
                    header=0,
                    engine='python',  # Hilft oft bei Parsing-Problemen
                    on_bad_lines='skip',  # Methode zum Überspringen von fehlerhaften Zeilen
                    encoding='utf-8')  # Falls Sonderzeichen vorhanden sind

    df_population = df_population[['City', 'Value', 'Year']]

    df_population.rename(columns={'Value': 'Population'}, inplace=True)

    df_population.dropna(inplace=True)
    df_population['Population'] = df_population['Population'].astype(int)

    output_path = './data/population_data.csv'
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    df_population.to_csv(output_path, index=False)
    print(f"✅ Datei wurde gespeichert: {output_path}")

    # Merge der beiden DataFrames basierend auf der "City"-Spalte
    df = pd.merge(df, df_population, on=['City', 'Year'], how='left')

    return(df)

In [ ]:
output_path = './data/population_data.csv'
os.makedirs(os.path.dirname(output_path), exist_ok=True)
df_population.to_csv(output_path, index=False)
print(f"✅ Datei wurde gespeichert: {output_path}")

In [ ]:
# Merge der beiden DataFrames basierend auf der "City"-Spalte
df_merged = pd.merge(df, df_population, on='City', how='left')

In [ ]:
# Funktion für Übersicht über dtypes, missing values, unique values etc.
def overview(df):
    '''
    Erstelle einen Überblick über einige Eigenschaften der Spalten eines DataFrames.
    VARs
        df: Der zu betrachtende DataFrame
    RETURNS:
        None
    '''
    display(pd.DataFrame({'dtype': df.dtypes,
                          'total': df.count(),
                          'missing': df.isna().sum(),
                          'missing%': df.isna().mean()*100,
                          'n_uniques': df.nunique(),
                          'uniques%': df.nunique()/df.shape[0]*100,
                          'uniques': [df[col].unique() for col in df.columns]
                         }))
overview(df_population)

In [ ]:
df_filtered_population = df_population[df_population['City'].isin(df['City'])][['Country or Area','City', 'Year', 'City type', 'Value']]
df_filtered_population['Value'] = df_filtered_population['Value'].astype(int)
df_filtered_population.head()